In [223]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import math

In [224]:
data =  pd.read_csv('combo.gzip',compression='gzip',parse_dates=True,index_col='date')

In [225]:
data['departure_delay'] = data['departure_delay'].astype(int)
data['arrival_delay'] = data['arrival_delay'].astype(int)

In [226]:
data = data.reset_index()

In [227]:
data.head()

,date,airline,airline_code,departure_airport,departure_state,departure_lat,departure_lon,arrival_airport,arrival_state,arrival_lat,arrival_lon,departure_schedule,departure_actual,departure_delay,arrival_schedule,arrival_actual,arrival_delay
0,2012-11-25,US,20355,CLT,NC,35.21,-80.94,PHL,PA,39.87,-75.24,1755.0,1805,10,1931.0,1948,17
1,2012-11-29,US,20355,CLT,NC,35.21,-80.94,PHL,PA,39.87,-75.24,1755.0,1752,-3,1931.0,1929,-2
2,2012-11-29,US,20355,CLT,NC,35.21,-80.94,PHL,PA,39.87,-75.24,1445.0,1442,-3,1623.0,1615,-8
3,2012-11-29,US,20355,CLT,NC,35.21,-80.94,PHL,PA,39.87,-75.24,2005.0,2007,2,2140.0,2139,-1
4,2012-11-29,US,20355,CLT,NC,35.21,-80.94,PHL,PA,39.87,-75.24,1630.0,1624,-6,1808.0,1805,-3


In [228]:
def makeDateTime(date,timeInInt):
    hour = math.floor(timeInInt//100)
    extraday = False
    if(hour==24):
        hour = 0
        extraday = True
        
    dateTime = datetime.datetime(date.year,date.month,date.day,hour,math.floor(timeInInt%100),0)
    
    if(extraday):
        dateTime = dateTime + datetime.timedelta(days = 1)
        
    return dateTime

In [229]:
def makeDateTimeWithDelay(dateTime, delayInMinutes):
    actual = dateTime
    delay = datetime.timedelta(minutes= math.floor(delayInMinutes))
    return actual-delay

In [233]:
data['departure_actual_datetime'] = np.nan
data['departure_schedule_datetime'] = np.nan
data['arrival_actual_datetime'] = np.nan
data['arrival_schedule_datetime'] = np.nan

In [239]:
def fillInRows(row):
    base_date = row['date']
    arrival_actual = row['arrival_actual']
    departure_actual = row['departure_actual']
    arrival_delay = row["arrival_delay"]
    departure_delay = row["departure_delay"]
    
    
    row["departure_actual_datetime"] = makeDateTime(base_date,departure_actual)
    row['arrival_actual_datetime'] = makeDateTime(base_date,arrival_actual)
    
    row['departure_schedule_datetime'] = makeDateTimeWithDelay(makeDateTime(base_date,departure_actual),departure_delay)
    row['arrival_schedule_datetime'] = makeDateTimeWithDelay(makeDateTime(base_date,arrival_actual),arrival_delay)
    return row


print(datetime.datetime.now())
data = data.apply(fillInRows,axis=1)
print(datetime.datetime.now())

2018-04-26 17:01:54.162591
2018-04-26 18:10:46.771159


In [241]:
data.tail()

,date,airline,airline_code,departure_airport,departure_state,departure_lat,departure_lon,arrival_airport,arrival_state,arrival_lat,...,departure_schedule,departure_actual,departure_delay,arrival_schedule,arrival_actual,arrival_delay,departure_actual_datetime,departure_schedule_datetime,arrival_actual_datetime,arrival_schedule_datetime
11401191,2012-03-01,AA,19805,ORD,IL,41.98,-87.9,DCA,VA,38.85,...,2020.0,2143,83,2300.0,16,76,2012-03-01 21:43:00,2012-03-01 20:20:00,2012-03-01 00:16:00,2012-02-29 23:00:00
11401192,2012-04-23,UA,19977,ORD,IL,41.98,-87.9,DCA,VA,38.85,...,705.0,702,-3,953.0,950,-3,2012-04-23 07:02:00,2012-04-23 07:05:00,2012-04-23 09:50:00,2012-04-23 09:53:00
11401193,2012-03-18,UA,19977,ORD,IL,41.98,-87.9,DCA,VA,38.85,...,1615.0,1612,-3,1906.0,1856,-10,2012-03-18 16:12:00,2012-03-18 16:15:00,2012-03-18 18:56:00,2012-03-18 19:06:00
11401194,2012-04-30,AA,19805,ORD,IL,41.98,-87.9,DCA,VA,38.85,...,640.0,635,-5,920.0,917,-3,2012-04-30 06:35:00,2012-04-30 06:40:00,2012-04-30 09:17:00,2012-04-30 09:20:00
11401195,2012-04-10,AA,19805,ORD,IL,41.98,-87.9,DCA,VA,38.85,...,640.0,659,19,920.0,947,27,2012-04-10 06:59:00,2012-04-10 06:40:00,2012-04-10 09:47:00,2012-04-10 09:20:00


In [244]:
for column in data:
    assert pd.notnull(data[column]).all()
    print(column+' is clean')

date is clean
airline is clean
airline_code is clean
departure_airport is clean
departure_state is clean
departure_lat is clean
departure_lon is clean
arrival_airport is clean
arrival_state is clean
arrival_lat is clean
arrival_lon is clean


AssertionError: 

In [245]:
data.to_csv('datetime.gzip',compression='gzip')